In [18]:
import glob
import netCDF4 as nc
import numpy as np
from rapidfuzz import process
import pandas as pd
import re
import xarray as xr
import geopandas as gpd
from functools import reduce
pd.set_option('display.max_rows', 20)
import os
os.chdir("/Users/caropark/FAO_ag_check_code/")


# crop indices

In [ ]:
i= '/Users/caropark/Dropbox (Harvard University)/DataByCountry/Portugal'
end_fao = '/fao_data.nc'
end_cal = '/crop_calendar_data_sage.nc'

fao = nc.Dataset(str(i)+end_fao)
crops = np.transpose(fao['crop']).tobytes()
blob = re.findall('[A-Z][^A-Z]*',  crops.decode('latin-1'))
fao_key = pd.DataFrame({"crop": blob}).reset_index().rename({"index": "fao_idx", "crop": "cropname"}, axis=1)
fao_key = fao_key.applymap(lambda x: x.strip() if isinstance(x, str) else x)

end1 = "/harvest_area_fraction_data_sage_all.nc"
area = xr.open_dataset(str(i)+end1)
crops2 = np.transpose(area['crop_FAO'].values).tobytes()
blob = re.findall('[A-Z][^A-Z]*',  crops2.decode('latin-1'))
area_key = pd.DataFrame({"crop": blob}).reset_index().rename({"index": "harv_area_idx", "crop": "cropname"}, axis=1)
area_key = area_key.applymap(lambda x: x.strip() if isinstance(x, str) else x)

calendar = xr.open_dataset(str(i)+end_cal)
crops2 = np.transpose(xr.open_dataset(str(i)+end_cal)['crop_FAO'].values).tobytes()
blob = re.findall('[A-Z][^A-Z]*',  crops2.decode('latin-1'))
cal_key = pd.DataFrame({"crop": blob}).reset_index().rename({"index": "crop",  "crop": "cropname"}, axis=1)
cal_key = cal_key.applymap(lambda x: x.strip() if isinstance(x, str) else x)

df = fao_key.merge(area_key, how="left")

cal_key.loc[cal_key['cropname']=="Cotton", 'cropname'] = "Seed cotton"
cal_key.loc[cal_key['cropname']=="Oat", 'cropname'] = "Oats"
cal_key.loc[cal_key['cropname']=="Pulses", 'cropname'] = "Pulses nes"
cal_key.loc[cal_key['cropname']=="Sugarbeets", 'cropname'] = "Sugar beet"
cal_key.loc[cal_key['cropname']=="Sunflower", 'cropname'] = "Sunflower seed"

df= df.merge(cal_key, how="left").dropna().reset_index()
df = cal_key.merge(df, how="left")
newkey = pd.read_csv("./data/cropkey_fao_update.csv")
df = newkey.merge(df, how="left")
df.to_pickle("./data/calendar_fao_cropkey.pkl")

# country names

In [2]:
## laod in disparate data

fao = pd.read_csv("./data/faostat_all_flags.csv")[['Area Code (ISO3)', 'Area']].drop_duplicates().reset_index(drop=True)
fao.columns=["iso_a3", "fao"]

lowres = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))[['name', 'iso_a3']].reset_index(drop=True).rename(columns={'name':'lowres'})
lowres.loc[lowres["lowres"].isin(["France", "Norway"]), "iso_a3"] = (lowres["lowres"].map({"France": "FRA", "Norway": "NOR"}))

wb = pd.read_csv("./data/wb_classification.csv", index_col=None)[['country', 'iso_a3']].rename(columns={'country': 'wb'})[:218] #remove the groups "SSA"
dfs = [fao, lowres, wb]
merged = reduce(lambda left, right: pd.merge(left, right, on="iso_a3", how="outer"), dfs)
missing = merged[merged.isna().any(axis=1)].sort_values('iso_a3').to_csv("./data/iso_a3_missing.csv") ## human check done here

In [3]:
## create lookup for iso_a3

lookup = (merged.melt(id_vars="iso_a3", value_vars=["fao", "lowres", "wb"],
                         value_name="match")
                     .dropna(subset=["match"])
                     .drop(columns="variable")
                     .drop_duplicates())

def normalize_name(name: str) -> str:
    if pd.isna(name):
        return None
    return (re.sub(r"\(.*?\)", "", name)  
              .lower()
              .replace("&", "and")
              .replace("st.", "saint")
              .replace("  ", " ")
              .strip())

lookup["norm_name"] = lookup["match"].map(normalize_name)
lookup = lookup.drop('match', axis=1).drop_duplicates()
choices = lookup["norm_name"].unique()

def best_match(name, choices):
    if pd.isna(name):
        return None, None
    match = process.extractOne(name, choices)
    return match[0], match[1] 

In [5]:
## load in TARGET!! 
mylist = [f for f in glob.glob("/Users/caropark/Harvard University Dropbox/Caro Park/CropYield_VI_Global/DataByCountry/*")]
angela_names = pd.DataFrame({'country': np.unique([path.rsplit('/', 1)[1] for path in mylist])})
angela_names = angela_names[angela_names['country']!="Icon\r"]
angela_names["norm_name"] = angela_names["country"].map(normalize_name)
angela_names[["best_match", "score"]] = (angela_names["norm_name"].apply(lambda n: pd.Series(best_match(n, choices))))
angela_names = angela_names.drop("norm_name", axis=1)

angela_iso = angela_names.merge(lookup, left_on="best_match", right_on="norm_name", how="left").drop_duplicates()[['country', 'iso_a3', 'score']]


In [10]:
## human check and adjustment

## first fix
angela_iso_100 = angela_iso.query(' score ==100 ')
mustfix_remove = angela_iso_100[angela_iso_100.duplicated(subset=["iso_a3"], keep=False)].reset_index(drop=True).loc[[1,2],]
angela_clean = angela_iso.merge(mustfix_remove, how="left", indicator=True)
angela_clean = angela_clean[angela_clean["_merge"] == "left_only"].drop(columns="_merge")

## second fix
angela_clean.query('score !=100 ')
angela_clean.loc[angela_clean["score"] != 100, "iso_a3"] = ["F351", np.nan, "CIV", "GUF", "REU", "TOK"]
angela_clean = angela_clean.dropna(subset="iso_a3").drop("score", axis=1)

,country,iso_a3
0,Afghanistan,AFG
1,Albania,ALB
2,Algeria,DZA
3,Angola,AGO
4,Antigua and Barbuda,ATG
...,...,...
202,Venezuela (Bolivarian Republic of),VEN
203,Viet Nam,VNM
204,Yemen,YEM
205,Zambia,ZMB


In [12]:
country_key = merged.merge(angela_clean, on= "iso_a3", how="outer")
country_key

,iso_a3,fao,lowres,wb,country
0,AFG,Afghanistan,Afghanistan,Afghanistan,Afghanistan
1,ALB,Albania,Albania,Albania,Albania
2,DZA,Algeria,Algeria,Algeria,Algeria
3,AGO,Angola,Angola,Angola,Angola
4,ATG,Antigua and Barbuda,NaN,Antigua and Barbuda,Antigua and Barbuda
...,...,...,...,...,...
230,MAF,NaN,NaN,St. Martin (French part),NaN
231,TCA,NaN,NaN,Turks and Caicos Islands,NaN
232,TUV,NaN,NaN,Tuvalu,Tuvalu
233,VIR,NaN,NaN,Virgin Islands (U.S.),NaN


In [15]:
country_key.to_csv("./data/country_key.csv", index=False)